In [8]:
import numpy as np
import pandas as pd
from itertools import combinations
import matplotlib.pyplot as plt
%matplotlib inline
import scipy as sc
import time
import sys
%cd /Users/James/Desktop/Eigth Semester/Stochastic Modelling/NFL_Project/Week Dependent Analysis

/Users/James/Desktop/Eigth Semester/Stochastic Modelling/NFL_Project/Week Dependent Analysis


In [9]:
# method
filesScore = "week_data_2017.csv"
filesTeams = "teams_2017.csv"

filesPercs = "2018_win_perc.csv"
namesCols = "win_perc_2017"

In [10]:
def week_UB_Pij(data, weekub, method, weighted):
    
    # Getting the Data in the right form
    data = data.drop(['game_id', 'home_team'], axis=1)
    if method == "less":
        # IF METHOD IS WEEKS LESS THAN:
        data["week"] = data[data["week"] < weekub]["week"]
    if method == "equal":
        # IF METHOD IS GETTING THE EXACT WEEK
        data["week"] = data[data["week"] == weekub]["week"]
    if method == "between":
        # If METHOD IS BETWEEN (WITH CONTAINMENT ON THE UPPER END)
        data["week"] = data[np.logical_and(data["week"] > weekub[0],data["week"] <= weekub[1])]["week"]
    data = data.dropna()
    data = data.drop("week", axis = 1)
    data_np = np.array(data)
    
    Pij = np.zeros((32,32))
    
    # Computing the Pij Matrix
    for iCnt in range(np.shape(data_np)[0]):
        winner = int(data_np[iCnt,0])-1
        loser =  int(data_np[iCnt,1])-1
        # checking if an interdivisional game has occurred
        if (Pij[loser,winner] == 0) & (Pij[winner,loser] == 0):
            Pij[winner,loser] = data_np[iCnt,2]
        # when the other team had won the first game
        elif (Pij[loser,winner] > 0):
            if (Pij[loser,winner] > data_np[iCnt,2]):
                Pij[loser,winner] = (Pij[loser,winner] - data_np[iCnt,2])/2
            else: 
                Pij[winner,loser] = -(Pij[loser,winner] - data_np[iCnt,2])/2
                Pij[loser,winner] = 0
        # when the same team won the first game
        else: 
            Pij[winner,loser] = (Pij[winner,loser] + data_np[iCnt,2])/2
            
    if(weighted == False):
                Pij[Pij!=0] = 1
            
    # making sure none of the columns sum to 0
    if len(np.where(Pij.sum(axis = 0) == 0)[0]) > 0:
        empty_vect = np.where(Pij.sum(axis = 0) == 0)[0]
        for iCnt in range(len(empty_vect)):
            Pij[empty_vect[iCnt],empty_vect[iCnt]] = 1
    
    # making the Pij matrix
    Pij = Pij/Pij.sum(axis=0)
        
    # returning the Pij Matrix
    return Pij

In [11]:
blocks = [0,4,8,12,16]
data = pd.read_csv('week_data_2017.csv',index_col=0)
alpha = 0.06

In [12]:
# making the teleporting probabilities uniform TO START
v = np.ones(32)/32
v = v[np.newaxis]
v = v.T


for iCnt in range(len(blocks)-1):
    # setting the weeks to look at
    weekub = [blocks[iCnt],blocks[iCnt+1]]
    
    # setting Pij
    Pij = week_UB_Pij(data, weekub, method = "between", weighted = True)
    # If we want the teleportation probabilties unifrom,
    # or based on team winning perc
                
    # initializing our first guess
    x0 = np.ones(32)/32
    x0 = x0[np.newaxis]
    x0 = x0.T
            
    # performing the simulation
    for i in range(5000):
        x0 = alpha*np.dot(Pij,x0) + (1-alpha)*v
        
    v = x0

    
scores = np.squeeze(x0)

In [13]:
teams = pd.read_csv('teams_2017.csv',index_col=0)
team_names = np.array(teams["team_name"])
print(team_names)

['NE' 'BUF' 'WAS' 'TEN' 'HOU' 'DET' 'CLE' 'CHI' 'CIN' 'LA' 'SF' 'GB' 'DAL'
 'MIN' 'DEN' 'BAL' 'TB' 'PIT' 'NO' 'KC' 'JAX' 'IND' 'CAR' 'OAK' 'LAC' 'SEA'
 'ATL' 'NYG' 'PHI' 'NYJ' 'ARI' 'MIA']


In [14]:
teamrank = {'Team': team_names, 'score': scores, 'wins': 32*16/2*scores}
teamrank_df = pd.DataFrame(data=teamrank)
teamrank_df = teamrank_df.sort_values(by=['score'], ascending = False)
print(teamrank_df)

   Team     score       wins
13  MIN  0.042130  10.785241
0    NE  0.039883  10.210143
18   NO  0.038831   9.940767
28  PHI  0.037527   9.607026
22  CAR  0.037464   9.590797
9    LA  0.037307   9.550687
19   KC  0.036020   9.221048
17  PIT  0.035203   9.011994
20  JAX  0.034924   8.940455
15  BAL  0.033949   8.690932
25  SEA  0.033526   8.582739
3   TEN  0.031676   8.109137
12  DAL  0.030933   7.918818
26  ATL  0.030919   7.915146
1   BUF  0.030296   7.755797
31  MIA  0.030155   7.719553
5   DET  0.029342   7.511624
2   WAS  0.029320   7.505966
7   CHI  0.029120   7.454845
4   HOU  0.028841   7.383346
11   GB  0.028622   7.327190
30  ARI  0.028522   7.301554
24  LAC  0.028229   7.226728
8   CIN  0.028041   7.178469
10   SF  0.027647   7.077744
29  NYJ  0.027583   7.061197
14  DEN  0.027289   6.985866
23  OAK  0.026340   6.742954
16   TB  0.025735   6.588088
21  IND  0.025138   6.435397
27  NYG  0.025089   6.422761
6   CLE  0.024398   6.245992
